# Install

In [5]:
pip install python-docx

   ---------------------------------------- 0.0/244.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/244.3 kB ? eta -:--:--
   ------ -------------------------------- 41.0/244.3 kB 653.6 kB/s eta 0:00:01
   -------------------------------------- - 235.5/244.3 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 244.3/244.3 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sympy as smp
from sympy import I, Add, Mul, N, cos, sin, simplify, expand, factor, collect
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum.dagger import Dagger
from docx import Document
from docx.shared import Inches
from itertools import product
import numpy as np

# Original Calculations

In [3]:
t = smp.symbols('t', real=True)

sig_i = smp.eye(2)
sig_x = smp.Matrix([[0, 1], [1, 0]])
sig_y = smp.Matrix([[0, -I], [I, 0]])
sig_z = smp.Matrix([[1, 0], [0, -1]])

In [4]:
def create_gamma(Like_list):
    gammas = []
    gammas.append(0.1)
    for element in Like_list:
        if element == "like":
            gammas.append(0.1)
        else:
            gammas.append(0.02)
    return gammas

In [5]:
def pauli_on_qubit(pauli_matrix, qubit_index, num_of_qubits):
    operators = [sig_i] * num_of_qubits
    operators[qubit_index] = pauli_matrix
    result = operators[0]
    for op in operators[1:]:
        result = TensorProduct(result, op)
    return result

In [6]:
def create_hamiltonian(num_of_qubits, gammas, distances):
    dim = 2 ** num_of_qubits
    hamiltonian = smp.zeros(dim, dim)
    for i in range(num_of_qubits):
        for j in range (i+1, num_of_qubits):
            if (gammas[i] == gammas[j]): # like spins
                h_int = gammas[i]*gammas[j]*(2*pauli_on_qubit(sig_z, i, num_of_qubits)*pauli_on_qubit(sig_z, j, num_of_qubits) - pauli_on_qubit(sig_x, i, num_of_qubits)*pauli_on_qubit(sig_x, j, num_of_qubits) - pauli_on_qubit(sig_y, i, num_of_qubits)*pauli_on_qubit(sig_y, j, num_of_qubits))/(4*distances[i][j]**3)
            else: # unlike spins
                h_int = gammas[i]*gammas[j]*pauli_on_qubit(sig_z, i, num_of_qubits)*pauli_on_qubit(sig_z, j, num_of_qubits)/(2*distances[i][j]**3)
            hamiltonian += h_int
    return hamiltonian

In [7]:
rho_x_polarized = smp.Rational(1, 2) * (sig_i + sig_x)
rho_y_polarized = smp.Rational(1, 2) * (sig_i + sig_y)
rho_z_polarized = smp.Rational(1, 2) * (sig_i + sig_z)
rho_u_polarized = smp.Rational(1, 2) * (sig_i)

def create_initial_density_matrix(polarization, num_of_qubits):
    if polarization[0] == 'x':
        rho = rho_x_polarized
    elif polarization[0] == 'y':
        rho = rho_y_polarized
    elif polarization[0] == 'z':
        rho = rho_z_polarized
    else:
        rho = rho_u_polarized

    for i in range(1, num_of_qubits):
        if polarization[i] == 'x':
            rho = TensorProduct(rho, rho_x_polarized)
        elif polarization[i] == 'y':
            rho = TensorProduct(rho, rho_y_polarized)
        elif polarization[i] == 'z':
            rho = TensorProduct(rho, rho_z_polarized)
        else:
            rho = TensorProduct(rho, rho_u_polarized)
    return rho

In [8]:
def partial_trace_multiple(rho, trace_qubits, num_of_qubits):
    """
    Perform the partial trace on the given density matrix `rho` over multiple specified qubits.

    Parameters:
    - rho: sympy.Matrix, the density matrix.
    - trace_qubits: list of int, the indices of the qubits to trace out (0-based).
    - num_of_qubits: int, the total number of qubits in the system.

    Returns:
    - sympy.Matrix, the reduced density matrix.
    """
    remaining_qubits = sorted(set(range(num_of_qubits)) - set(trace_qubits))
    num_remaining_qubits = len(remaining_qubits)
    reduced_dim = 2 ** num_remaining_qubits
    
    # Initialize the reduced density matrix
    reduced_rho = smp.zeros(reduced_dim, reduced_dim)
    
    # Map indices in the reduced space to indices in the full space
    for i in range(reduced_dim):
        for j in range(reduced_dim):
            full_indices_i = list(bin(i)[2:].zfill(num_remaining_qubits))
            full_indices_j = list(bin(j)[2:].zfill(num_remaining_qubits))
            
            trace_sum = 0
            for traced_bits in range(2 ** len(trace_qubits)):
                trace_indices = list(bin(traced_bits)[2:].zfill(len(trace_qubits)))
                
                full_i = [None] * num_of_qubits
                full_j = [None] * num_of_qubits
                
                for qi, ri in enumerate(remaining_qubits):
                    full_i[ri] = full_indices_i[qi]
                    full_j[ri] = full_indices_j[qi]
                
                for ti, qi in enumerate(trace_qubits):
                    full_i[qi] = trace_indices[ti]
                    full_j[qi] = trace_indices[ti]
                
                full_i_idx = int(''.join(full_i), 2)
                full_j_idx = int(''.join(full_j), 2)
                
                trace_sum += rho[full_i_idx, full_j_idx]
            
            reduced_rho[i, j] = trace_sum
    
    return reduced_rho

In [20]:
def create_evolve_expect(gammas, distances, polarization, num_of_qubits, central_qubit=0):
    # create
    hamiltonian = create_hamiltonian(num_of_qubits, gammas, distances)
    rho_total_0 = create_initial_density_matrix(polarization, num_of_qubits)
    
    # evolve
    U = smp.exp(-I * hamiltonian * t)
    rho_total_t = U * rho_total_0 * Dagger(U)
    rho_0_t = partial_trace_multiple(rho_total_t, [i for i in range(num_of_qubits) if i!=central_qubit], num_of_qubits)

    # expect
    expectation_Sx = smp.trace(sig_x * rho_0_t).simplify()
    expectation_Sy = smp.trace(sig_y * rho_0_t).simplify()
    expectation_Sz = smp.trace(sig_z * rho_0_t).simplify()
    return (expectation_Sx, expectation_Sy, expectation_Sz)


In [21]:
# Define a function to ignore terms smaller than a threshold
def filter_small_terms(expr, threshold=1e-3):
    if expr.is_Atom:
        # For atomic values (numbers, symbols), check their magnitude
        if abs(expr) < threshold:
            return 0
        else:
            return expr
    elif expr.is_Add:
        # Process addition terms
        return Add(*[filter_small_terms(arg, threshold) for arg in expr.args])
    elif expr.is_Mul:
        # Process multiplication terms
        coeff, factors = expr.as_coeff_mul()
        if abs(coeff) < threshold:
            return 0
        return Mul(coeff, *[filter_small_terms(f, threshold) for f in factors])
    else:
        return expr

# Testing

In [ ]:
# 3 qubits
gammas = [0.1, 0.1, 0.02]
distances = [["NA", 1, 1], [1, "NA", 1], [1, 1, "NA"]]
polarization = ['y','z','z']
num_of_qubits = 3

expression = create_evolve_expect(gammas, distances, polarization, num_of_qubits, central_qubit=0)
filtered_exp = tuple(filter_small_terms(e) for e in expression)
rounded_num = tuple(N(e, 2) for e in filtered_exp)
rounded_num_sincos = tuple(simplify(factor(expand(e).rewrite(sin))) for e in rounded_num)

final_expr = tuple(simplify(e) for e in rounded_num_sincos)
print(final_expr)

(-0.25*sin(0.007*t) - 0.25*sin(0.007*t) - 0.5*sin(0.017*t) + 0.25*I*cos(0.007*t) - 0.25*I*cos(0.007*t), 0.5*cos(0.007*t) + 0.5*cos(0.017*t), 0.5 - 0.5*cos(0.01*t))


# Make Documents

In [55]:
def check_is_needed(distance_matrix,gammas,state):
    list_1 = ['uuu', 'xuu', 'xyu','xyz','xxy', 'xxz', 'xxu','xxx','yuu','yyx','yyz','yyu','yyy','zuu','zxu','zyu','zzu','zzx','zzy','zzz']
    list_2 = ['uuu', 'xuu', 'uux', 'xyu', 'uxy', 'yux', 'xyz', 'xzy', 'yzx', 'xxy', 'xyx', 'xxz', 'xzx', 'xxu', 'xux', 'xxx', 'yuu', 'uuy', 'yyx', 'yxy', 'yyz', 'yzy', 'yyu', 'yuy', 'yyy', 'zuu', 'uuz', 'zxu', 'zux', 'xuz', 'zyu', 'zuy', 'yuz', 'zzu', 'zuz', 'zzx', 'zxz', 'zzy', 'zyz', 'zzz']
    list_3 = ['uuu', 'xuu', 'uxu', 'xyu', 'yux', 'uxy', 'xyz', 'yxz', 'zxy', 'xxy', 'yxx', 'xxz', 'zxx', 'xxu', 'uxx', 'xxx', 'yuu', 'uyu', 'yyx', 'xyy', 'yyz', 'zyy', 'yyu', 'uyy', 'yyy', 'zuu', 'uzu', 'zxu', 'xzu', 'uzx', 'zyu', 'yzu', 'uzy', 'zzu', 'uzz', 'zzx', 'xzz', 'zzy', 'yzz', 'zzz']


    if distance_matrix[0][1]==1 and distance_matrix[0][2]==1 and distance_matrix[1][2]==1:
        if gammas[1] == 0.1 and gammas[2] == 0.1: #Like Like
            if state in list_1:
                return 1
        elif gammas[1] == 0.1 and gammas[2] == 0.02:
            if state in list_2:
                return 1
        elif gammas[1] == 0.02 and gammas[2] == 0.02:
            if state in list_3:
                return 1
    elif distance_matrix[0][1]==100 and distance_matrix[0][2]==100 and distance_matrix[1][2]==100:
        # if gammas[1] == 0.1 and gammas[2] == 0.1:
        #     if state in list_1:
        #         return 1
        # elif gammas[1] == 0.1 and gammas[2] == 0.02:
        #     if state in list_2:
        #         return 1
        # elif gammas[1] == 0.02 and gammas[2] == 0.02:
        #     if state in list_3:
        #         return 1
        return 0
    elif distance_matrix[0][1]==1 and distance_matrix[0][2]==100 and distance_matrix[1][2]==100:
        if gammas[1] == 0.1 and gammas[2] == 0.1:
            if state in list_2:
                return 1
        elif gammas[1] == 0.1 and gammas[2] == 0.02:
            if state in list_2:
                return 0
        elif gammas[1] == 0.02 and gammas[2] == 0.02:
            return 1
    elif distance_matrix[0][1]==100 and distance_matrix[0][2]==100 and distance_matrix[1][2]==1:
        if gammas[1] == 0.1 and gammas[2] == 0.1:
            if state in list_3:
                return 0
        elif gammas[1] == 0.1 and gammas[2] == 0.02:
            return 0
        elif gammas[1] == 0.02 and gammas[2] == 0.02:
            if state in list_3:
                return 1
    elif distance_matrix[0][1]==100 and distance_matrix[0][2]==1 and distance_matrix[1][2]==100:
        if gammas[1] == 0.1 and gammas[2] == 0.1:
            return 0
        elif gammas[1] == 0.1 and gammas[2] == 0.02:
            return 0
        elif gammas[1] == 0.02 and gammas[2] == 0.02:
            return 0
    return 0

In [56]:
def generate_combinations(letters, n):
    combinations = [''.join(comb) for comb in product(letters, repeat=n)]
    return combinations

In [57]:
def computation(number_of_spins):
    letters = ['x','y', 'z', 'u']
    state_of_spins = generate_combinations(letters, number_of_spins)
    
    distance_matrix_1 = [[0,1,1],[1,0,1],[1,1,0]] # [1,1,1]
    distance_matrix_2 = [[0,100,100],[100,0,100],[100,100,0]] # [100,100,100]
    distance_matrix_3 = [[0,1,100],[1,0,100],[100,100,0]] # [1,100,100]
    distance_matrix_4 = [[0,100,100],[100,0,1],[100,1,0]] # [100,100,1]
    distance_matrix_5 = [[0,100,1],[100,0,100],[1,100,0]] # [100,1,100]
    distance_matrices = [distance_matrix_1,distance_matrix_2,distance_matrix_3,distance_matrix_4,distance_matrix_5]
    
    Like_1 = ["like","like"]
    Like_2 = ["like","unlike"]
    Like_3 = ["unlike","unlike"]
    Like_lists = [Like_1,Like_2,Like_3]
    
    for distance_matrix in distance_matrices:
        doc = Document()
        doc.add_heading('Data for analysis', level=1)
        for Like_list in Like_lists:
            for state in state_of_spins:
                gammas = create_gamma(Like_list)
                is_needed = check_is_needed(distance_matrix, gammas, state)
                if is_needed==0:
                    continue
                filepath = f"correct images/Distances_{distance_matrix[0][1]}_{distance_matrix[0][2]}_{distance_matrix[1][2]}/{Like_list[0]}_{Like_list[1]}/spin_expectation_values_Distances {distance_matrix[0][1]} {distance_matrix[0][2]} {distance_matrix[1][2]}; S1 {Like_list[0]}, S2 {Like_list[1]}; State {state[0]} {state[1]} {state[2]}.png"
                print(filepath)
                polarization = [state[0], state[1], state[2]]
                num_of_qubits = 3
                text = []
                for i in range(num_of_qubits):
                    expression = create_evolve_expect(gammas, distance_matrix, polarization, num_of_qubits, central_qubit=i)
                    filtered_exp = tuple(filter_small_terms(e) for e in expression)
                    rounded_num = tuple(N(e, 2) for e in filtered_exp)
                    rounded_num_sincos = tuple(simplify(factor(expand(e).rewrite(sin))) for e in rounded_num)
                    final_expr = tuple(simplify(e) for e in rounded_num_sincos)
                    text.append(final_expr)
                doc.add_heading(f'Section: Distances {distance_matrix[0][1]} {distance_matrix[0][2]} {distance_matrix[1][2]}; S1 {Like_list[0]}, S2 {Like_list[1]}; State {state[0]} {state[1]} {state[2]}', level=2)
                paragraph = doc.add_paragraph('The spin expectation values for each spin are:\n')
                for i in range(num_of_qubits):
                    paragraph.add_run(f'Spin {i}:\n\n').bold = True
                    for j in range(0, 3):
                        if j == 0:
                            paragraph.add_run(f'Sx = {text[i][j]} \n\n')
                        elif j == 1:
                            paragraph.add_run(f'Sy = {text[i][j]} \n\n')
                        elif j == 2:
                            paragraph.add_run(f'Sz = {text[i][j]} \n\n')
                doc.add_picture(filepath, width=Inches(7))
                doc.add_page_break()
                print(f"Distances {distance_matrix[0][1]} {distance_matrix[0][2]} {distance_matrix[1][2]}; S1 {Like_list[0]}, S2 {Like_list[1]}; State {state[0]} {state[1]} {state[2]}")
                                # rho0 = create_initial_density_matrix(number_of_spins,state)
                                # plot_function(rho0, gammas, thetas, distance_matrix, number_of_spins,text)
        document_name = f"Distances_{distance_matrix[0][1]}_{distance_matrix[0][2]}_{distance_matrix[1][2]}.docx"
        doc.save(document_name)

In [58]:
computation(3)

correct images/Distances_1_1_1/like_like/spin_expectation_values_Distances 1 1 1; S1 like, S2 like; State x x x.png
Distances 1 1 1; S1 like, S2 like; State x x x
correct images/Distances_1_1_1/like_like/spin_expectation_values_Distances 1 1 1; S1 like, S2 like; State x x y.png
Distances 1 1 1; S1 like, S2 like; State x x y
correct images/Distances_1_1_1/like_like/spin_expectation_values_Distances 1 1 1; S1 like, S2 like; State x x z.png
Distances 1 1 1; S1 like, S2 like; State x x z
correct images/Distances_1_1_1/like_like/spin_expectation_values_Distances 1 1 1; S1 like, S2 like; State x x u.png
Distances 1 1 1; S1 like, S2 like; State x x u
correct images/Distances_1_1_1/like_like/spin_expectation_values_Distances 1 1 1; S1 like, S2 like; State x y z.png
Distances 1 1 1; S1 like, S2 like; State x y z
correct images/Distances_1_1_1/like_like/spin_expectation_values_Distances 1 1 1; S1 like, S2 like; State x y u.png
Distances 1 1 1; S1 like, S2 like; State x y u
correct images/Distanc